In [28]:
import pandas as pd
import numpy as np
import logging
from pathlib import Path
from typing import Literal, Optional

import pandas as pd
from entities import EM_CDS_TRACKER_DICT, FX_TRACKER_DICT
from utils import Backtest, load_trackers, clean_stale_prices
from pathlib import Path
from aamalha.utils.performance import Performance

# Trackers Statistics

In [ ]:
fx = load_trackers(FX_TRACKER_DICT).rename(columns=lambda col: col + "_fx")
cds = load_trackers(EM_CDS_TRACKER_DICT).rename(columns=lambda col: col + "_cds")

trackers = pd.concat(
    [fx, cds],
    axis=1,
).fillna(method="ffill")
trackers = pd.concat(
    [clean_stale_prices(trackers[col].copy()) for col in trackers], axis=1
)
trackers = trackers[:"2024-09-30"]
trackers

In [15]:
(trackers / trackers.apply(lambda col: col[col.first_valid_index()]) - 1).filter(
    like="_cds", axis=1
).sort_index(axis=1).rename(
    columns=lambda col: col[:3]
).to_clipboard(excel=True)

In [3]:
perf_trackers = Performance(trackers)

In [17]:
df_perf = perf_trackers.table.T.copy()
DICT_COLS = {
    "Return": "Retorno",
    "Vol": "Volatilidade",
    "Skew": "Assimetria",
    "Kurt": "Curtose",
    "Sharpe": "Sharpe",
    "Sortino": "Sortino",
    "DD 1%q": "DD (1%)",
    "Max DD": "Max DD",
    "Start Date": "Início",
}
df_perf.rename(
    columns=DICT_COLS, index=lambda idx: f"{idx[:3]} ({idx[4:].upper()})"
).to_clipboard()

In [ ]:
perf_trackers.rolling_sharpe.ARS_cds

In [5]:
DICT_COLS = {
    "count": "N obs",
    "mean": "Média",
    "std": "DP",
    "min": "Min",
    "max": "Max",
}
trackers_rolling_sharpe = perf_trackers.rolling_sharpe.describe().T
trackers_rolling_sharpe["Início"] = perf_trackers.rolling_sharpe.apply(
    lambda col: col.first_valid_index()
)
trackers_rolling_sharpe.rename(
    columns=DICT_COLS, index=lambda idx: f"{idx[:3]} ({idx[4:].upper()})"
).to_clipboard()

# Valor

In [ ]:
FOLDER = Path(
    "C:/Users/pcampos/OneDrive - Insper - Instituto de Ensino e Pesquisa/Dissertação Mestrado/Analysis/backtests"
)
list_series = []
for file_path in FOLDER.glob("*VALUE-*.xlsx"):
    file_path.is_file()
    s_backtest = pd.read_excel(file_path, index_col=0)["backtest"]
    s_backtest.name = file_path.stem
    list_series.append(s_backtest)

df_backtest = pd.concat(
    [clean_stale_prices(col.copy()) for col in list_series], axis=1
).sort_index(ascending=True)
df_backtest = df_backtest.dropna(how="all")
df_backtest.iloc[0].loc[df_backtest.iloc[1].isna()] = np.nan
df_backtest

In [19]:
from bwbbgdl import GoGet
import statsmodels.api as sm


tickers = ["SPXT Index", "FXCTEM8 Index"]
g = GoGet(enforce_strict_matching=True)
benchmarks: pd.DataFrame = g.fetch(
    tickers=tickers,
    fields="PX_LAST",
)
benchmarks = benchmarks.pivot_table(index="date", columns="id")
benchmarks.columns = benchmarks.columns.droplevel(0)

N_DAYS = 1
exog = benchmarks.fillna(method="ffill").copy()
endog = np.log(df_backtest["VALUE-FX-PAIRED-12"]).diff(N_DAYS).dropna()
exog = np.log(exog).diff(N_DAYS).reindex(endog.dropna().index).fillna(method="ffill")
exog = sm.add_constant(exog)
model = sm.OLS(endog, exog).fit()
print(model.summary())

In [ ]:
from bwbbgdl import GoGet
import statsmodels.api as sm


tickers = ["SPXT Index", "FXCTEM8 Index", "MXWD Index", "HYG Equity", "EREM5LD5 Index"]
g = GoGet(enforce_strict_matching=True)
benchmarks: pd.DataFrame = g.fetch(
    tickers=tickers,
    fields="PX_LAST",
)
benchmarks = benchmarks.pivot_table(index="date", columns="id")
benchmarks.columns = benchmarks.columns.droplevel(0)

N_DAYS = 1
exog = benchmarks.fillna(method="ffill").copy()
endog = np.log(df_backtest["VALUE-FX-PAIRED-12"]).diff(N_DAYS).dropna()
exog = np.log(exog).diff(N_DAYS).reindex(endog.dropna().index).fillna(method="ffill")
exog = sm.add_constant(exog)
model = sm.OLS(endog, exog).fit()
print(model.summary())

In [20]:
df_backtest.to_clipboard(excel=True)


In [ ]:
df_backtest

## Backtests

In [22]:
performance_1y = Performance(df_backtest)
performance_5y = Performance(df_backtest, rolling_window=252*5)

## Estatísticas Descritivas dos  Trackers - Retornos Diários									

In [23]:
DICT_COLS = {
    "Return": "Retorno",
    "Vol": "Volatilidade",
    "Skew": "Assimetria",
    "Kurt": "Curtose",
    "Sharpe": "Sharpe",
    "Sortino": "Sortino",
    "DD 1%q": "DD (1%)",
    "Max DD": "Max DD",
    "Start Date": "Início",
}
performance_1y.table.T.rename(columns=DICT_COLS).to_clipboard()

## Sharpe Média Móvel 1 Ano														

In [24]:
DICT_COLS = {
    "count": "N",
    "mean": "Média",
    "std": "DP",
    "min": "Min",
    "max": "Max",
}

performance_1y_rolling_sharpe = performance_1y.rolling_sharpe.describe().T
performance_1y_rolling_sharpe["Início"] = performance_1y.rolling_sharpe.apply(
    lambda col: col.first_valid_index()
)
performance_1y_rolling_sharpe.rename(columns=DICT_COLS).to_clipboard()

In [26]:
rolling_sharpe = performance_1y.rolling_sharpe.copy()

N = 500
percentils = [n / N for n in range(0, N + 1)]
col = performance_1y.rolling_sharpe.iloc[:, 0]
pd.concat(
    [
        pd.Series(
            {p: rolling_sharpe[col].quantile(p) for p in percentils},
            name=col,
        )
        for col in rolling_sharpe.columns
    ],
    axis=1,
).to_clipboard()

## Sharpe Média Móvel 5 Ano

In [25]:
DICT_COLS = {
    "count": "N",
    "mean": "Média",
    "std": "DP",
    "min": "Min",
    "max": "Max",
}

performance_5y_rolling_sharpe = performance_5y.rolling_sharpe.describe().T
performance_5y_rolling_sharpe["Início"] = performance_5y.rolling_sharpe.apply(
    lambda col: col.first_valid_index()
)
performance_5y_rolling_sharpe.rename(columns=DICT_COLS).to_clipboard()

In [27]:
# rolling_sharpe_quantiles

rolling_sharpe = performance_5y.rolling_sharpe.copy()

N = 500
percentils = [n / N for n in range(0, N +1)]
col = performance_5y.rolling_sharpe.iloc[:, 0]
pd.concat(
    [
        pd.Series(
            {p: rolling_sharpe[col].quantile(p) for p in percentils},
            name=col,
        )
        for col in rolling_sharpe.columns
    ],
    axis=1,
).to_clipboard()

# Long Only

In [29]:
FOLDER = Path(
    "C:/Users/pcampos/OneDrive - Insper - Instituto de Ensino e Pesquisa/Dissertação Mestrado/Analysis/backtests"
)
files = [
    "L-CDS-EW",
    "L-CDS-IV",
    "L-FX-EW",
    "L-FX-IV",
    "L-EW",
    "L-IV",
]


In [ ]:
FOLDER = Path(
    "C:/Users/pcampos/OneDrive - Insper - Instituto de Ensino e Pesquisa/Dissertação Mestrado/Analysis/backtests"
)
files = [
    "L-CDS-EW",
    "L-CDS-IV",
    "L-FX-EW",
    "L-FX-IV",
    "L-EW",
    "L-IV",
]
file_pathS = [FOLDER.joinpath(f"{file}.xlsx") for file in files]
list_series = []
for file_path in file_pathS:
    file_path.is_file()
    s_backtest = pd.read_excel(file_path, index_col=0)["backtest"]
    s_backtest.name = file_path.stem
    list_series.append(s_backtest)

df_backtest = pd.concat(
    [clean_stale_prices(col.copy()) for col in list_series], axis=1
).sort_index(ascending=True)
df_backtest = df_backtest["2009-02-03":]
df_backtest = df_backtest.dropna(how="all")
df_backtest.iloc[0].loc[df_backtest.iloc[1].isna()] = np.nan
df_backtest

In [51]:
df_backtest.to_clipboard(excel=True)


In [ ]:
df_backtest

## Backtests

In [37]:
performance_1y = Performance(df_backtest)
performance_5y = Performance(df_backtest, rolling_window=252*5)

## Estatísticas Descritivas dos  Trackers - Retornos Diários									

In [50]:
DICT_COLS = {
    "Return": "Retorno",
    "Vol": "Volatilidade",
    "Skew": "Assimetria",
    "Kurt": "Curtose",
    "Sharpe": "Sharpe",
    "Sortino": "Sortino",
    "DD 1%q": "DD (1%)",
    "Max DD": "Max DD",
    "Start Date": "Início",
}
performance_1y.table.T.rename(columns=DICT_COLS).to_clipboard()

## Sharpe Média Móvel 1 Ano														

In [49]:
DICT_COLS = {
    "count": "N",
    "mean": "Média",
    "std": "DP",
    "min": "Min",
    "max": "Max",
}

performance_1y_rolling_sharpe = performance_1y.rolling_sharpe.describe().T
performance_1y_rolling_sharpe["Início"] = performance_1y.rolling_sharpe.apply(
    lambda col: col.first_valid_index()
)
performance_1y_rolling_sharpe.rename(columns=DICT_COLS).to_clipboard()

In [47]:
rolling_sharpe = performance_1y.rolling_sharpe.copy()

N = 500
percentils = [n / N for n in range(0, N + 1)]
col = performance_1y.rolling_sharpe.iloc[:, 0]
pd.concat(
    [
        pd.Series(
            {p: rolling_sharpe[col].quantile(p) for p in percentils},
            name=col,
        )
        for col in rolling_sharpe.columns
    ],
    axis=1,
).to_clipboard()

## Sharpe Média Móvel 5 Ano

In [45]:
DICT_COLS = {
    "count": "N",
    "mean": "Média",
    "std": "DP",
    "min": "Min",
    "max": "Max",
}

performance_5y_rolling_sharpe = performance_5y.rolling_sharpe.describe().T
performance_5y_rolling_sharpe["Início"] = performance_5y.rolling_sharpe.apply(
    lambda col: col.first_valid_index()
)
performance_5y_rolling_sharpe.rename(columns=DICT_COLS).to_clipboard()

In [44]:
rolling_sharpe = performance_5y.rolling_sharpe.copy()

N = 500
percentils = [n / N for n in range(0, N +1)]
col = performance_5y.rolling_sharpe.iloc[:, 0]
pd.concat(
    [
        pd.Series(
            {p: rolling_sharpe[col].quantile(p) for p in percentils},
            name=col,
        )
        for col in rolling_sharpe.columns
    ],
    axis=1,
).to_clipboard()

# Long Short- Classe

In [ ]:
FOLDER = Path(
    "C:/Users/pcampos/OneDrive - Insper - Instituto de Ensino e Pesquisa/Dissertação Mestrado/Analysis/backtests"
)

file_pathS = [FOLDER.joinpath(f"{file}.xlsx") for file in files]
list_series = []
for file_path in list(FOLDER.glob("L-*-S-*")):
    file_path.is_file()
    s_backtest = pd.read_excel(file_path, index_col=0)["backtest"]
    s_backtest.name = file_path.stem
    list_series.append(s_backtest)

df_backtest = pd.concat(
    [clean_stale_prices(col.copy()) for col in list_series], axis=1
).sort_index(ascending=True)
df_backtest = df_backtest["2009-02-03":]
df_backtest = df_backtest.dropna(how="all")
df_backtest.iloc[0].loc[df_backtest.iloc[1].isna()] = np.nan
df_backtest

In [57]:
df_backtest.to_clipboard(excel=True)


## Backtests

In [58]:
performance_1y = Performance(df_backtest)
performance_5y = Performance(df_backtest, rolling_window=252*5)

## Estatísticas Descritivas dos  Trackers - Retornos Diários									

In [59]:
DICT_COLS = {
    "Return": "Retorno",
    "Vol": "Volatilidade",
    "Skew": "Assimetria",
    "Kurt": "Curtose",
    "Sharpe": "Sharpe",
    "Sortino": "Sortino",
    "DD 1%q": "DD (1%)",
    "Max DD": "Max DD",
    "Start Date": "Início",
}
performance_1y.table.T.rename(columns=DICT_COLS).to_clipboard()

## Sharpe Média Móvel 1 Ano														

In [60]:
DICT_COLS = {
    "count": "N",
    "mean": "Média",
    "std": "DP",
    "min": "Min",
    "max": "Max",
}

performance_1y_rolling_sharpe = performance_1y.rolling_sharpe.describe().T
performance_1y_rolling_sharpe["Início"] = performance_1y.rolling_sharpe.apply(
    lambda col: col.first_valid_index()
)
performance_1y_rolling_sharpe.rename(columns=DICT_COLS).to_clipboard()

In [61]:
rolling_sharpe = performance_1y.rolling_sharpe.copy()

N = 500
percentils = [n / N for n in range(0, N + 1)]
col = performance_1y.rolling_sharpe.iloc[:, 0]
pd.concat(
    [
        pd.Series(
            {p: rolling_sharpe[col].quantile(p) for p in percentils},
            name=col,
        )
        for col in rolling_sharpe.columns
    ],
    axis=1,
).to_clipboard()

## Sharpe Média Móvel 5 Ano

In [62]:
DICT_COLS = {
    "count": "N",
    "mean": "Média",
    "std": "DP",
    "min": "Min",
    "max": "Max",
}

performance_5y_rolling_sharpe = performance_5y.rolling_sharpe.describe().T
performance_5y_rolling_sharpe["Início"] = performance_5y.rolling_sharpe.apply(
    lambda col: col.first_valid_index()
)
performance_5y_rolling_sharpe.rename(columns=DICT_COLS).to_clipboard()

In [63]:
rolling_sharpe = performance_5y.rolling_sharpe.copy()

N = 500
percentils = [n / N for n in range(0, N + 1)]
col = performance_5y.rolling_sharpe.iloc[:, 0]
pd.concat(
    [
        pd.Series(
            {p: rolling_sharpe[col].quantile(p) for p in percentils},
            name=col,
        )
        for col in rolling_sharpe.columns
    ],
    axis=1,
).to_clipboard()

# Long Short - Ativo

In [64]:
FOLDER = Path(
    "C:/Users/pcampos/OneDrive - Insper - Instituto de Ensino e Pesquisa/Dissertação Mestrado/Analysis/backtests"
)
files = [
    "LS-CDS-FX-BN-IV",
    "LS-FX-CDS-BN-IV",
    "LS-FX-CDS-IV",
    "LS-FX-CDS-IV",
]
file_pathS = [FOLDER.joinpath(f"{file}.xlsx") for file in files]
list_series = []
for file_path in file_pathS:
    file_path.is_file()
    s_backtest = pd.read_excel(file_path, index_col=0)["backtest"]
    s_backtest.name = file_path.stem
    list_series.append(s_backtest)

df_backtest = pd.concat(
    [clean_stale_prices(col.copy()) for col in list_series], axis=1
).sort_index(ascending=True)
df_backtest = df_backtest["2009-02-03":]
df_backtest = df_backtest.dropna(how="all")
df_backtest.iloc[0].loc[df_backtest.iloc[1].isna()] = np.nan
df_backtest

,LS-CDS-FX-BN-IV,LS-FX-CDS-BN-IV,LS-FX-CDS-IV,LS-FX-CDS-IV
date,,,,
2009-02-03,88.732922,113.952739,98.793667,98.793667
2009-02-04,88.554396,114.184762,99.166558,99.166558
2009-02-05,88.344497,114.460641,99.509981,99.509981
2009-02-06,88.338092,114.473919,99.609150,99.609150
2009-02-09,88.363991,114.439778,99.719530,99.719530
...,...,...,...,...
2023-08-29,216.528809,-13.534421,32.890527,32.890527
2023-08-30,215.913695,-13.573725,33.051291,33.051291
2023-08-31,216.452711,-13.540070,32.809051,32.809051


In [65]:
df_backtest.to_clipboard(excel=True)


## Backtests

In [ ]:
performance_1y = Performance(df_backtest)
performance_5y = Performance(df_backtest, rolling_window=252*5)

## Estatísticas Descritivas dos  Trackers - Retornos Diários									

In [ ]:
DICT_COLS = {
    "Return": "Retorno",
    "Vol": "Volatilidade",
    "Skew": "Assimetria",
    "Kurt": "Curtose",
    "Sharpe": "Sharpe",
    "Sortino": "Sortino",
    "DD 1%q": "DD (1%)",
    "Max DD": "Max DD",
    "Start Date": "Início",
}
performance_1y.table.T.rename(columns=DICT_COLS).to_clipboard()

## Sharpe Média Móvel 1 Ano														

In [ ]:
DICT_COLS = {
    "count": "N",
    "mean": "Média",
    "std": "DP",
    "min": "Min",
    "max": "Max",
}

performance_1y_rolling_sharpe = performance_1y.rolling_sharpe.describe().T
performance_1y_rolling_sharpe["Início"] = performance_1y.rolling_sharpe.apply(
    lambda col: col.first_valid_index()
)
performance_1y_rolling_sharpe.rename(columns=DICT_COLS).to_clipboard()

In [ ]:
rolling_sharpe = performance_1y.rolling_sharpe.copy()

N = 500
percentils = [n / N for n in range(0, N + 1)]
col = performance_1y.rolling_sharpe.iloc[:, 0]
pd.concat(
    [
        pd.Series(
            {p: rolling_sharpe[col].quantile(p) for p in percentils},
            name=col,
        )
        for col in rolling_sharpe.columns
    ],
    axis=1,
).to_clipboard()

## Sharpe Média Móvel 5 Ano

In [ ]:
DICT_COLS = {
    "count": "N",
    "mean": "Média",
    "std": "DP",
    "min": "Min",
    "max": "Max",
}

performance_5y_rolling_sharpe = performance_5y.rolling_sharpe.describe().T
performance_5y_rolling_sharpe["Início"] = performance_5y.rolling_sharpe.apply(
    lambda col: col.first_valid_index()
)
performance_5y_rolling_sharpe.rename(columns=DICT_COLS).to_clipboard()